In [ ]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2023-08-28 07:36:30--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  --.-KB/s    in 0.07s   

2023-08-28 07:36:30 (69.5 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [ ]:
import pandas as pd
import numpy as np
import nltk

from nltk import word_tokenize

In [ ]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_csv("bbc_text_cls.csv")

In [ ]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [ ]:
# populate word2idx
# convert documents into sequences of ints / ids / indices
idx = 0
word2idx = {}
tokenized_docs = []
for doc in df["text"]:
  words = word_tokenize(doc.lower())
  doc_as_int = []
  for word in words:
    if word not in word2idx:
      word2idx[word] = idx
      idx += 1

    # save for later
    doc_as_int.append(word2idx[word])
  tokenized_docs.append(doc_as_int)

In [ ]:
# reverse mapping
# if you do it smarter you can store it as a list
idx2word = {v:k for k, v in word2idx.items()}

In [ ]:
# number of documents
N = len(df["text"])

In [ ]:
# number of words
V = len(word2idx)

In [ ]:
# instantiate term-frequency matrix
# note: could have also used count vectorizer
tf = np.zeros((N, V))

In [ ]:
# populate term-frequency counts
for i, doc_as_int in enumerate(tokenized_docs):
  for j in doc_as_int:
    tf[i, j] += 1

In [ ]:
# compute IDF
document_freq = np.sum(tf > 0, axis=0) # document frequency (shape = (V,))
idf = np.log(N / document_freq)

In [ ]:
# compute tf-idf
tf_idf = tf * idf

In [ ]:
np.random.seed(123)

In [ ]:
# pick a random document, show the top 5 terms (in terms of tf-idf score)
i = np.random.choice(N)
row = df.iloc[i]
print("Label: ", row["labels"])
print("Text: ", row["text"].split("\n", 1)[0])
print("Top 5 terms:")

scores = tf_idf[i]
indices= (-scores).argsort()

for j in indices[:5]:
  print(idx2word[j])

Label:  sport
Text:  Hingis hints at playing comeback
Top 5 terms:
hingis
pattaya
thailand
95th
30th


In [ ]:
# Exercise: use CountVectorizer to form the counts instead

# Exercise (hard): use Scipy's csr_matrix instead
# You cannot use X[i, j] += 1 here